In [17]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from google.colab import drive
import zipfile

In [18]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# Extract Dataset
zip_path = "/content/drive/MyDrive/LIL/TLR.zip"
extract_path = "/content/TLR"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [26]:
# Load Dataset
def load_dataset(dataset_path, img_size=(64, 64)):
    images, labels = [], []
    for class_label in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_label)
        # Check if the path is a directory and contains files
        if os.path.isdir(class_path) and len(os.listdir(class_path)) > 0:
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                # Check if the image was loaded correctly
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, img_size)
                    images.append(img)
                    labels.append(class_label)
                else:
                    print(f"Warning: Could not load image {img_path}") # Print a warning message
        else:
            print(f"Warning: Directory '{class_label}' is empty or not a directory")  # Inform about empty directories
    return np.array(images), np.array(labels)

In [21]:
# Preprocess Dataset
def preprocess_data(images, labels):
    images = images / 255.0
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    return images, labels

In [22]:
# Define CNN Model
def build_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Main Execution
if __name__ == "__main__":
    dataset_path = "/content/TLR"  # Set extracted dataset path
    images, labels = load_dataset(dataset_path)
    images, labels = preprocess_data(images, labels)

    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

    model = build_model(X_train.shape[1:], y_train.shape[1])
    # Allow user to specify epochs manually
    epochs = int(input("Enter the number of epochs: "))
    model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Evaluate Model
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {test_acc:.2f}")

In [ ]:
 # Save Model
    model.save("traffic_sign_model.h5")